Resnet Model Architecture

In [4]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Define the ResNet block
def resnet_block(x, filters, kernel_size=3, stride=1):
    y = layers.Conv2D(filters, kernel_size=kernel_size, strides=stride, padding='same')(x)
    y = layers.BatchNormalization()(y)
    y = layers.Activation('relu')(y)
    y = layers.Conv2D(filters, kernel_size=kernel_size, strides=1, padding='same')(y)
    y = layers.BatchNormalization()(y)

    # Shortcut connection
    if stride > 1:
        x = layers.Conv2D(filters, kernel_size=1, strides=stride, padding='same')(x)
        x = layers.BatchNormalization()(x)

    y = layers.add([x, y])
    y = layers.Activation('relu')(y)
    return y

# Define the ResNet model
def build_resnet_model(input_shape=(224, 224, 3), num_classes=1, count=0):  
    inputs = tf.keras.Input(shape=input_shape)

    # Initial convolution and max pooling
    x = layers.Conv2D(64, 7, strides=2, padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.MaxPooling2D(pool_size=3, strides=2, padding='same')(x)

    # Residual blocks
    x = resnet_block(x, 64, stride=1)
    x = resnet_block(x, 64, stride=1)
    x = resnet_block(x, 128, stride=2)
    x = resnet_block(x, 128, stride=1)
    x = resnet_block(x, 256, stride=2)
    x = resnet_block(x, 256, stride=1)
    x = resnet_block(x, 512, stride=2)
    x = resnet_block(x, 512, stride=1)
    
    # Global average pooling and fully connected layer
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(num_classes, activation='sigmoid')(x)

    model = models.Model(inputs, x, name='resnet_model')
    
    if count < 5:
        count += 1
        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
        return model
    else:
        return model

# Create and compile the ResNet model
resnet_model = build_resnet_model()
resnet_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Display model summary
resnet_model.summary()


Model: "resnet_model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d_20 (Conv2D)          (None, 112, 112, 64)         9472      ['input_2[0][0]']             
                                                                                                  
 batch_normalization_20 (Ba  (None, 112, 112, 64)         256       ['conv2d_20[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 activation_17 (Activation)  (None, 112, 112, 64)         0         ['batch_normalizati

Hyper parameter tuning and finding the model's best parameter

In [5]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from kerastuner.tuners import RandomSearch

# Define your data directory
data_dir = "SplitData"

# Set up data augmentation
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Create training and validation generators
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',  # Assuming it's binary classification (rot or not rot)
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

# Define ResNet model with hyperparameters as variables
def build_resnet_model(hp):
    base_model = tf.keras.applications.ResNet50(
        include_top=False,
        weights=None,
        input_shape=(224, 224, 3)
    )
    x = layers.GlobalAveragePooling2D()(base_model.output)
    
    # Tune the number of units in the dense layer
    hp_units = hp.Int('units', min_value=32, max_value=256, step=32)
    x = layers.Dense(units=hp_units, activation='relu')(x)

    output = layers.Dense(1, activation='sigmoid')(x)
    
    model = models.Model(inputs=base_model.input, outputs=output)

    # Compile the model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model

# Define the tuner and search space
tuner = RandomSearch(
    build_resnet_model,
    objective='val_accuracy',
    max_trials=5,  # You can adjust the number of trials
    directory='hyperparameter_tuning',
    project_name='basil_stem_rot'
)

# Search for the best hyperparameters
tuner.search(
    train_generator,
    epochs=5,  # Adjust as needed
    validation_data=validation_generator
)

# Get the best model and hyperparameters
best_model = tuner.get_best_models(num_models=1)[0]
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]

# Display the summary of the best model and hyperparameters
best_model.summary()
print("Best Hyperparameters:", best_hyperparameters.values)

# Save the best model
best_model.save('best_basil_stem_rot_model.h5')

Using TensorFlow backend
Found 805 images belonging to 3 classes.
Found 201 images belonging to 3 classes.


C:\Users\kaise\AppData\Local\Temp\ipykernel_81632\2485991668.py:4: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


Reloading Tuner from hyperparameter_tuning\basil_stem_rot\tuner0.json
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                    

Train the model (Resnet)

In [22]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from kerastuner.tuners import RandomSearch

# Define your data directory
data_dir = "SplitData"

# Set up data augmentation
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Create training and validation generators
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',  # Assuming it's binary classification (rot or not rot)
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

# Define ResNet model with tuned hyperparameters
def build_resnet_model(hp):
    base_model = tf.keras.applications.ResNet50(
        include_top=False,
        weights=None,
        input_shape=(224, 224, 3)
    )
    x = layers.GlobalAveragePooling2D()(base_model.output)
    
    # Use tuned hyperparameters
    hp_units = hp.get('units')
    x = layers.Dense(units=hp_units, activation='relu')(x)

    output = layers.Dense(1, activation='sigmoid')(x)
    
    model = models.Model(inputs=base_model.input, outputs=output)

    # Compile the model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model

# Load the best hyperparameters obtained from tuning
best_hyperparameters = {'units': 256}  # Replace this with the actual tuned hyperparameters

# Build the model with the best hyperparameters
tuned_model = build_resnet_model(best_hyperparameters)

# Train the model
tuned_history = tuned_model.fit(
    train_generator,
    epochs=50,  # Adjust as needed
    validation_data=validation_generator
)

# Save the tuned model
tuned_model.save('tuned_basil_stem_rot_model.h5')


Found 805 images belonging to 3 classes.
Found 201 images belonging to 3 classes.
Epoch 1/50
26/26 [==============================] - 215s 7s/step - loss: 0.2906 - accuracy: 0.8199 - val_loss: 0.0169 - val_accuracy: 0.8507
Epoch 2/50
26/26 [==============================] - 189s 7s/step - loss: -1.4782 - accuracy: 0.8422 - val_loss: 0.5844 - val_accuracy: 0.8507
Epoch 3/50
26/26 [==============================] - 191s 7s/step - loss: -3.2337 - accuracy: 0.8497 - val_loss: 0.0014 - val_accuracy: 0.8507
Epoch 4/50
26/26 [==============================] - 190s 7s/step - loss: -4.1588 - accuracy: 0.8497 - val_loss: 0.6112 - val_accuracy: 0.8507
Epoch 5/50
26/26 [==============================] - 188s 7s/step - loss: -9.2935 - accuracy: 0.8497 - val_loss: 4410.4326 - val_accuracy: 0.8507
Epoch 6/50
26/26 [==============================] - 189s 7s/step - loss: -15.4421 - accuracy: 0.8497 - val_loss: 183.1306 - val_accuracy: 0.8507
Epoch 7/50
26/26 [==============================] - 189s 7s/s